<a href="https://colab.research.google.com/github/pati-dev/text-multi-label/blob/master/attributes_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boot

In [1]:
%reload_ext tensorboard
%tensorflow_version 2.x
import datetime
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
finalDF = pd.read_csv('/content/drive/My Drive/Search Project/finalDF.csv')

Create text to sequence matrices

In [0]:
num_words = 1000
seq_len = 30

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(finalDF.text)
sequences = tokenizer.texts_to_sequences(finalDF.text)
x = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=seq_len)

In [5]:
y = np.array(finalDF.iloc[:,2:])

print(x.shape)
print(y.shape)

(1880, 30)
(1880, 8)


Prepare data

In [0]:
y = y.astype('float32')
x_train, x_tv, y_train, y_tv = train_test_split(x, y, test_size=0.3, random_state=6969)
x_valdn, x_test, y_valdn, y_test = train_test_split(x_tv, y_tv, test_size=0.3, random_state=6969)

In [0]:
# pathToLogs = '/content/drive/My Drive/Search Project/logs/'
pathToLogs = '/content/logs/'

# Clear any logs from previous runs
!rm -rf {pathToLogs}

Fit model

In [0]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=num_words, 
                                    output_dim=100, 
                                    input_length=seq_len,
                                    dtype='float32'))
# model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.LSTM(units=32))
model.add(tf.keras.layers.LayerNormalization())
# model.add(tf.keras.layers.Dropout(0.1))

# model.add(tf.keras.layers.Conv1D(filters=32, 
#                                  kernel_size=2,
#                                  strides=1,
#                                  activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(units=1024, 
#                                activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=32, 
                               activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(units=y_train.shape[1], 
                               activation='sigmoid'))

In [32]:
learning_rate = 0.001

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
valdn_log_dir = 'logs/gradient_tape/' + current_time + '/valdn'

train_writer = tf.summary.create_file_writer(train_log_dir)
valdn_writer = tf.summary.create_file_writer(valdn_log_dir)


# Sigmoid Cross-entropy with Logits
def computeLoss(labels, logits):
  return tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)


# Exact Match Ratio or Subset Accuracy
def computeAccuracy(truth, predict):
  binaryPred = np.where(predict > 0.5, 1., 0.)
  acc = accuracy_score(y_true=truth, y_pred=binaryPred, normalize=True)
  return acc


def train_step(features, labels, optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)):
  with tf.GradientTape() as tape:
    
    # Compute loss
    logits = model(features, training=True)
    loss = computeLoss(labels, logits)
    loss_value = tf.reduce_mean(loss)
    
    # Compute accuracy
    predictions = model(x_train, training=True)
    acc = computeAccuracy(y_train, predictions)

  # Compute gradients and minimize using the optimizer
  grads = tape.gradient(loss_value, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  return loss_value, acc


def test_step(model, features, labels):
  # Compute loss
  predictions = model(features)
  loss = computeLoss(labels, predictions)
  loss_value = tf.reduce_mean(loss)
  # Compute accuracy
  acc = computeAccuracy(labels, predictions)

  return loss_value, acc


def train(epochs, batchSize):
  
  # Epoch loop
  for epoch in range(epochs):
    
    # Initialize vars
    epoch_loss = 0
    epoch_acc = 0
    numBatches = math.ceil(x_train.shape[0] / batchSize)
    
    # Batch loop
    for batch in range(numBatches):
      
      # Compute indices
      start = batch*batchSize
      end = (batch+1)*batchSize
      
      # Extract batch
      features, labels = x_train[start:end], y_train[start:end]
      
      # Train
      batch_loss, batch_acc = train_step(features, labels)
      
      # Update local loss and accuracy
      epoch_loss += batch_loss
      epoch_acc += batch_acc
    
    # Update global loss and accuracy
    train_loss = epoch_loss / numBatches
    train_acc = epoch_acc / numBatches

    # Log training loss and accuracy
    with train_writer.as_default():
      tf.summary.scalar('training loss', train_loss, step=epoch)
      tf.summary.scalar('accuracy', train_acc, step=epoch)

    # Validate the model
    valdn_loss, valdn_acc = test_step(model, x_valdn, y_valdn)

    # Log validation loss and accuracy
    with valdn_writer.as_default():
      tf.summary.scalar('training loss', valdn_loss, step=epoch)
      tf.summary.scalar('accuracy', valdn_acc, step=epoch)
    
    # Pretty print output
    template = 'Epoch {}, Training Loss: {}, Training Accuracy: {}, Validation Loss: {}, Validation Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss,
                          train_acc*100,
                          valdn_loss,
                          valdn_acc*100))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 100)           100000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                17024     
_________________________________________________________________
layer_normalization (LayerNo (None, 32)                64        
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
Total params: 118,536
Trainable params: 118,472
Non-trainable params: 64
_________________________________________________

In [33]:
train(20, 50)

Epoch 1, Training Loss: 0.6565086245536804, Training Accuracy: 1.5704154002026338, Validation Loss: 0.6312613487243652, Validation Accuracy: 2.7918781725888326
Epoch 2, Training Loss: 0.6213921904563904, Training Accuracy: 5.507711358775188, Validation Loss: 0.6256600618362427, Validation Accuracy: 2.7918781725888326
Epoch 3, Training Loss: 0.6003577709197998, Training Accuracy: 10.29494540132838, Validation Loss: 0.6255175471305847, Validation Accuracy: 4.060913705583756
Epoch 4, Training Loss: 0.5826834440231323, Training Accuracy: 15.827986040751995, Validation Loss: 0.6307210326194763, Validation Accuracy: 2.5380710659898478
Epoch 5, Training Loss: 0.5683370232582092, Training Accuracy: 22.790723854553647, Validation Loss: 0.6293677091598511, Validation Accuracy: 2.030456852791878
Epoch 6, Training Loss: 0.5573309659957886, Training Accuracy: 28.97388269728695, Validation Loss: 0.6151149868965149, Validation Accuracy: 4.822335025380711
Epoch 7, Training Loss: 0.5467313528060913, Tr

In [0]:
%tensorboard --logdir logs/gradient_tape
notebook.display(port=6006, height=800)

In [0]:
trainPred = model.predict(x_train)
trainPred[:5]

In [0]:
metrics = model.evaluate(x_test, y_test)
print('test loss, test acc:', metrics)

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/My Drive/Search Project/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Search Project/model.h5")
print("Saved model to disk")

In [0]:
accuracy_score(np.array([[0, 1, 1],
                         [1, 1, 0],
                         [1, 1, 1],
                         [1, 1, 1],
                         [0, 0, 1]]),
               np.array([[0, 1, 1],
                         [0, 1, 1],
                         [0, 0, 0],
                         [1, 1, 0],
                         [0, 0, 1]]))*100

40.0